In [321]:
import pandas as pd
import numpy as np

In [322]:
data = pd.read_csv("data/indicator_data/BEL-USD.csv")

In [323]:
data.head()

,Date,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2020-09-16,6.171814,6.244687,4.075332,4.608443,58191896,-1.264755e+08,105996915,-0.568734,-9.789113e+07,...,-91.052855,0.000000,6.054850,0.0,-5.408562,-1.081712,-4.326850,-26.741449,-31.117521,-26.741449
1,2020-09-17,4.585522,4.921655,3.020856,3.020856,20733476,-1.472090e+08,85263439,-0.605514,-8.860899e+07,...,-100.000000,0.000000,5.919202,0.0,-12.285064,-3.322383,-8.962681,-34.449530,-42.234981,-51.978676
2,2020-09-18,3.045186,3.854492,2.853754,3.001233,14830439,-1.576683e+08,70433000,-0.611248,-7.599214e+07,...,-97.945980,0.000000,5.790403,0.0,-18.816058,-6.421118,-12.394940,-0.649578,-0.651697,-52.290611
3,2020-09-19,3.019029,5.558504,2.987119,4.594291,74813134,-1.389617e+08,145246134,-0.417606,-4.811017e+07,...,-75.758610,0.000000,5.693580,0.0,-20.204764,-9.177847,-11.026917,53.080119,42.579125,-26.966411
4,2020-09-20,4.639493,5.006308,3.481535,3.685932,21665054,-1.548183e+08,123581080,-0.436818,-4.404866e+07,...,-88.409815,-0.579884,5.598443,0.0,-25.332520,-12.408782,-12.923738,-19.771478,-22.029110,-41.406231


In [324]:
LOOKBACK = 20
FEATURE_NUMBER = 87
complete_data = pd.DataFrame()

columns = np.delete(data.columns, [0, 4])
target = "Close"
new_df = data[columns].sample(n=FEATURE_NUMBER, axis='columns')
new_df["Close"] = data["Close"]
for index, row in new_df.iterrows():
    try:
        tmp_df = new_df.iloc[index:index+LOOKBACK,:]
        tmp_X = tmp_df.iloc[:,:FEATURE_NUMBER]
        tmp_y = new_df.iloc[index+LOOKBACK,-1]
        tmp_X = tmp_X.to_numpy().flatten()
        tmp_X = pd.Series(tmp_X)
        tmp_y = pd.Series(tmp_y)
        tmp_var = tmp_X.append(tmp_y, ignore_index=True)
        tmp_var = tmp_var.values.reshape(1, len(tmp_var.values))
        tmp_var = pd.DataFrame(tmp_var)
        tmp_var.columns = [*tmp_var.columns[:-1], 'Close']
        complete_data = complete_data.append(tmp_var, ignore_index=True)
    except:
        continue

complete_data["Close"] = complete_data["Close"].pct_change()
complete_data = complete_data.dropna(axis=0)
complete_data["Close"] = complete_data["Close"].apply(lambda x: 'Buy' if x > 0 else 'Sell')
complete_data

,0,1,2,3,4,5,6,7,8,9,...,1731,1732,1733,1734,1735,1736,1737,1738,1739,Close
1,0.745458,8.208521,-2.270394e+07,6.527316,12.0,6.041451,-2.807709,4.0,0.0,0.000000,...,1.697345,-3.262070,-4.622800e+06,-256.581957,0.662626,-84.0,205.995900,3.300288,5.537924,Sell
2,0.844712,7.215698,-7.238920e+06,6.443765,16.0,6.927154,-2.398021,4.0,0.0,0.020540,...,1.594624,-4.513562,-3.980495e+06,-318.959707,0.603250,-88.0,217.258134,3.284977,5.522613,Buy
3,0.762602,7.162830,3.961457e+07,6.443765,16.0,18.455142,-2.470820,4.0,0.0,0.242414,...,1.522898,-4.824086,-3.343233e+06,-359.071425,0.557445,-88.0,227.527120,3.284046,5.521682,Buy
4,0.815097,6.899475,3.542740e+07,6.443765,16.0,17.902011,-2.367959,4.0,0.0,0.115902,...,1.466025,-4.471098,-2.843530e+06,-395.762506,0.516736,-88.0,218.730897,3.007948,5.521682,Sell
5,0.886301,6.515167,-6.301568e+06,6.443765,16.0,17.458540,-2.203857,4.0,0.0,0.027544,...,1.400160,-4.375466,-2.520454e+06,-424.753545,0.484528,-84.0,168.810605,2.440114,5.521682,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,0.967887,1.249129,-1.801426e+06,1.107421,4.0,41.300732,0.019532,84.0,0.0,0.392071,...,0.845009,0.539435,-1.159826e+05,-122.886754,0.065204,-92.0,30.805308,0.879955,1.009490,Buy
569,0.980043,1.241477,-2.550908e+05,1.082827,4.0,36.898531,-0.032106,80.0,0.0,0.139618,...,0.834923,0.243423,-6.926484e+04,-126.580434,0.062779,-92.0,31.034408,0.878575,1.009490,Buy
570,1.049247,1.212774,-4.878913e+05,1.082827,8.0,38.651291,0.023621,76.0,0.0,0.236892,...,0.827813,1.686565,-4.651240e+04,-130.815761,0.059357,-88.0,22.864425,0.843299,1.009490,Sell
571,1.078933,1.179093,8.399700e+04,1.082827,4.0,33.224487,0.081852,72.0,0.0,0.101476,...,0.810349,2.633569,-1.431664e+05,-144.179080,0.062432,-96.0,27.521372,0.825136,0.997300,Sell


In [325]:
from sklearn.model_selection import train_test_split

X = complete_data.iloc[:,:-1]
y = complete_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [326]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=100)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [327]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

0.5217391304347826
[[31 22]
 [33 29]]


In [328]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

param_grid = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"],
    "max_depth": [1, 10, 100, 150, 250, 500]
    # "min_samples_split": [1, 10, 100, 150, 250, 500],
    # "min_samples_leaf": [1, 10, 100, 150, 250, 500],
    # "max_features": ["auto", "sqrt", "log2"],
    }
criterion = ["gini", "entropy"]
splitter = ["best", "random"]
max_depth = [1, 10, 100, 150, 250, 500, 1000, 10000]

best_acc = 0
best_params = []

for p1 in criterion:
    for p2 in splitter:
        for p3 in max_depth:
            model = DecisionTreeClassifier(criterion=p1, splitter=p2, max_depth=p3)
            model.fit(X_train, y_train)
            train_preds = model.predict(X_train)
            test_preds = model.predict(X_test)
            # print(p1, p2, p3)
            # print("Train acc: ", accuracy_score(y_train, train_preds))
            # print("Test acc: ", accuracy_score(y_test, test_preds))
            if accuracy_score(y_test, test_preds) > best_acc:
                del best_params
                print(p1,p2,p3)
                best_acc = accuracy_score(y_test, test_preds)
                print("Test acc: ", best_acc)
                best_params = [p1, p2, p3]
                print()
print(best_acc)
print(best_params)

gini best 1
Test acc:  0.46956521739130436

gini best 10
Test acc:  0.5043478260869565

gini best 100
Test acc:  0.5130434782608696

gini best 500
Test acc:  0.5217391304347826

gini best 1000
Test acc:  0.5391304347826087

gini random 100
Test acc:  0.5478260869565217

entropy best 10
Test acc:  0.6

entropy best 500
Test acc:  0.6086956521739131

entropy random 150
Test acc:  0.6434782608695652

0.6434782608695652
['entropy', 'random', 150]
